간단한 상황
- Weapon : 50
- Target : 100 (weight : 1~100)
- 적재량 : 3

- 상황 구현
- 에이전트 구현

kernel은 base

목적함수 대신, 새로운 결과치를 내야 한다.
1개 요격 시 보상인데, 오래 걸렸을 수록 감쇠된 보상을 얻어야 한다.
그런데 위험도에 따라 보상이 달라져야겠지.
그럼 weight 만큼의 보상을 주면 되지 않을까

np.random.seed(42)는 요격확률과 초기 위험도를 설정하게 해 줄것이다.


random.seed(42)는 요격시 실제로 요격되는지에 설정할 것이다.

In [69]:
# 필요 라이브러리 import
import numpy as np
import copy
import random

In [159]:
# WTA World 클래스
class WTA_World():
    def __init__(self,W,T,M):
        self.W = W
        self.T = T
        self.M = M

        # Target t에 weapon w를 할당할 때의 요격확률 생성
        self.PK_wt = np.random.rand(W, T)
        # Target index 할당
        self.targets = np.array(list(range(T)))
        # Target 별 초기 위험도 할당
        self.init_weights = np.random.randint(1,100,size=T)
        # Target의 현재 위험도 설정 (0 = 파괴)
        self.weights = copy.copy(self.init_weights)


        self.M_W = [M] * W
        self.left_M_W = self.M_W
        
    def fire(self, theta_wt):
        # 값이 1인 요소를 찾기
        indices = np.where(theta_wt == 1)
        
        # 좌표 형태로 출력
        coordinates = list(zip(indices[0], indices[1]))

        for coord in coordinates :
            w,t = coord
            if self.PK_wt[w][t] > 0 and self.left_M_W[w] > 0 :
                pk_probability = self.PK_wt[w][t]
                random_value = random.random()
                if random_value <  pk_probability :
                    # 요격 성공
                    self.weights[coord[1]] = 0
                else : 
                    # 요격 실패
                    pass
                self.left_M_W[w] -= 1
            else:
                pass
        
        done = self.is_done()

        return self.weights, self.left_M_W, done

    def is_done(self):
        if sum(self.left_M_W) == 0 or sum(self.weights) == 0 :
            return True
        else :
            return False
        
    def get_state(self):
        return (self.PK_wt, self.weights, self.left_M_W)
    
    def reset(self):
        pass

In [ ]:
# Greedy Algorithm

def greedy(state):
    PK_wt = state[0]
    targets_weights = state[1]
    left_M_W = state[2]
    
    w = PK_wt.shape[0]
    t = PK_wt.shape[1]
    
    # w의 발사 여부
    m_w = [1 if m > 0 else m for m in left_M_W]
    # 할당 여부를 나타내는 theta_wt
    theta_wt = np.zeros((w, t), dtype=int)

    flattened = (PK_wt*targets_weights).flatten()
    sorted_indices = np.argsort(flattened)[::-1]
    
    i = 0
    indices_list = []
    while True :
        if i == w * t :
            break
        v = sorted_indices[i]
        row = v // T # Weapon
        col = v % T # Target

        if left_M_W[row] > 0 and m_w[row] == 1:
            
            m_w[row] -= 1
            theta_wt[row][col] = 1
            indices_list.append(v)
            print(f"weapon {row}를 target {col}에 할당, 요격확률*가중치:{flattened[v]:.2f} ")

        # 할당할 수 있는 유도탄의 개수를 모두 소모했다면 종료.
        if sum(left_M_W) == 0:
            break

        i += 1


    return theta_wt

In [184]:
wta=WTA_World(50,100,20)
step_i = 1
while True : 
    print(f"{step_i}번째 스텝")
    theta_wt =greedy(wta.get_state())
    print("------발사------")
    wta.fire(theta_wt)
    print("------요격결과------")
    print(wta.get_state()[1])
    if wta.is_done():
        break
    step_i+=1

1번째 스텝
weapon 38를 target 51에 할당, 요격확률*가중치:95.29 
weapon 0를 target 87에 할당, 요격확률*가중치:94.52 
weapon 14를 target 51에 할당, 요격확률*가중치:93.86 
weapon 8를 target 64에 할당, 요격확률*가중치:93.64 
weapon 30를 target 87에 할당, 요격확률*가중치:93.16 
weapon 40를 target 65에 할당, 요격확률*가중치:92.78 
weapon 23를 target 51에 할당, 요격확률*가중치:92.41 
weapon 18를 target 45에 할당, 요격확률*가중치:92.12 
weapon 31를 target 51에 할당, 요격확률*가중치:92.12 
weapon 20를 target 87에 할당, 요격확률*가중치:92.11 
weapon 44를 target 23에 할당, 요격확률*가중치:91.55 
weapon 17를 target 23에 할당, 요격확률*가중치:91.50 
weapon 35를 target 29에 할당, 요격확률*가중치:91.22 
weapon 39를 target 65에 할당, 요격확률*가중치:91.18 
weapon 16를 target 64에 할당, 요격확률*가중치:90.97 
weapon 15를 target 51에 할당, 요격확률*가중치:90.90 
weapon 6를 target 29에 할당, 요격확률*가중치:90.74 
weapon 46를 target 45에 할당, 요격확률*가중치:90.74 
weapon 45를 target 64에 할당, 요격확률*가중치:89.64 
weapon 2를 target 65에 할당, 요격확률*가중치:89.59 
weapon 13를 target 45에 할당, 요격확률*가중치:89.45 
weapon 9를 target 57에 할당, 요격확률*가중치:89.41 
weapon 42를 target 23에 할당, 요격확률*가중치:88.93 
weapon 43를 target 87에 할당, 요격확률*가